<a href="https://colab.research.google.com/github/csmaster23/machine_learning_principles/blob/main/Auto_Budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
# Connect your google drive that houses the csv files
month_year = "September 2024"
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Budgets/Auto_Budgets/2024/September/'  # Replace with your file path
csv_name = 'Chase_Credit_Sept_2024.CSV'
path = os.path.join(file_path, csv_name)
print(path)

In [ ]:
# Load csv where each row is a dictionary with the appropriate key words for each row from the column names
def load_csv(file_path):
  data = []
  with open(file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      data.append(row)
  return data
print(f"Loading {path}...")
data = load_csv(path)
print("Finished.")

In [ ]:
# Find all the potential column names from the uploaded csv
keys = data[0].keys()
print(keys)

In [ ]:
# Get important columns
amounts, category, descr = [], [], []
for row in data:
  if float(row['Amount']) > 0: # ignore payments and returns (positive transactions for the credit card)
    continue
  amounts.append(row['Amount'])
  category.append(row['Category'])
  descr.append(row['Description'])
print(len(amounts), len(category), len(descr))
assert len(amounts) == len(category) == len(descr)

In [ ]:
# Create a dictionary to store the sum of amounts per category
category_totals = {}
for i in range(len(amounts)):
    cat = category[i]
    amt = float(amounts[i]) * (-1) # flip from negatives to positive values
    if cat in category_totals:
        category_totals[cat] += amt
    else:
        category_totals[cat] = amt

# Print the totals per category
for cat, total in category_totals.items():
    print(f"{cat}: {total}")

In [ ]:
# Create bar plot for spending by category
categories = list(category_totals.keys())
totals = list(category_totals.values())

plt.figure(figsize=(8, 4))
plt.bar(categories, totals, color='skyblue')
plt.title(f'Total Spending by Category - {month_year}')
plt.xlabel('Category')
plt.ylabel('Total Amount ($)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
plt.pie(totals, labels=categories, autopct='%1.1f%%', colors=plt.cm.Paired.colors)
plt.title(f'Spending Breakdown by Category - {month_year}')
plt.tight_layout()
plt.show()

In [ ]:
# Prompt user with actual spending before asking for projection
projections = {}
for cat in categories:
    actual = category_totals[cat]
    print(f"Your actual spending for {cat} in {month_year} was ${actual:.2f}.")
    projection = float(input(f"What was your projected spending for {cat}? $"))
    projections[cat] = projection

In [ ]:
# Calculate the differences between actuals and projections
differences = {cat: category_totals[cat] - projections[cat] for cat in categories}

# Define colors based on whether the user went over or under their projection
colors = ['green' if differences[cat] <= 0 else 'red' for cat in categories]

# Create the plot showing the difference between actual and projected spending
plt.figure(figsize=(6, 4))

# Bar plot for differences (positive: over, negative: under)
x = range(len(categories))
diff_values = list(differences.values())

plt.bar(categories, diff_values, color=colors)

# Customize plot
plt.title(f'Difference Between Actual and Projected Spending - {month_year}')
plt.xlabel('Category')
plt.ylabel('Difference ($)')
plt.axhline(0, color='black', linewidth=1)  # Add horizontal line at 0 for visual clarity
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for stacked bar plot
actuals = [category_totals[cat] for cat in categories]
projections_list = [projections[cat] for cat in categories]

plt.figure(figsize=(6, 4))

# Stacked bars: projected amount in light green and actual in skyblue
plt.bar(categories, projections_list, color='blue', label='Projected', align='center')
plt.bar(categories, actuals, color='red', label='Actual', align='center', alpha=0.6)

plt.title(f'Actual vs Projected Spending (Stacked) - {month_year}')
plt.xlabel('Category')
plt.ylabel('Amount ($)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()